In [ ]:
from __future__ import division
from psychopy.visual import ImageStim, TextStim, Window
from psychopy import core, event, gui, data, logging
import numpy as np
import pandas as pd
import os

from routines import Routine

# Code for the (simplified) reinforcement learning task from Fontanesi and colleagues (https://doi.org/10.3758/s13423-018-1554-2)

#general settings
expName = 'Group A'
screen_size = [800, 600]
frames_per_second = 60
full_screen = False
background_color = '#eeeeee'

# trial settings
choice_keys = ['q', 'p']
escape_key = 'escape'
choice_time_limit = 5
feedback_duration = 2
fixation_duration = 1

#stimuli settings
text_color = 'black'
text_height = 50
options_x_offset = 200
image_size = 100

#store info about the experiment session
dlg = gui.Dlg(title=expName)
dlg.addField('Participant:', 1)
dlg.addField('Age:', 25)
dlg.addField('Gender:', choices=['female', 'male', 'prefer not to disclose'])
dlg.addField("condition", choices = ["left", "right"]) #change into A and B
dlg.addField('Handedness:', choices=['right', 'left', 'both'])
dlg.show()

expInfo = dict(zip(['participant', 'age', 'gender', 'hand'], dlg.data))
expInfo['date'] = data.getDateStr()  # add a simple timestamp
expInfo['expName'] = expName # add the experiment name
if dlg.OK:  # then the user pressed OK
    print(expInfo)
else:
    print(expInfo)
    core.quit()

#check if data folder exists
directory=os.path.join(os.getcwd(), 'data')
if not os.path.exists(directory):
    os.makedirs(directory)

#create file name for storing data
fileName = os.path.join('data', '%s_%s_%s' % (expName, expInfo['participant'], expInfo['date']))

#save a log file
logFile = logging.LogFile(fileName + '.log', level=logging.DEBUG)
logging.console.setLevel(logging.WARNING)  # this outputs to the screen, not a file

#create a window
mywin = Window(screen_size, units='pix', color=background_color, fullscr=full_screen)

#create some stimuli


#Lotterien (2x2 outcomes, ähnlichem EV, verschiedene Varianz)
#Zufällige Zahlen 1 bis 10 x 30
a = np.random.choice(a = np.arange(1,11), size = 30, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px)
b = np.random.choice(a = np.arange(0.3, 0.7, 0.05), size = 30, replace = True)

#EVx: x * px
c = a * b

# Y = evx/py
d = c / (1 - b)

#py: 1 - px
e = 1 - b

#Evy: y * py
f = d * e

#creating data frame for nornal Lotteries
x = {"x": a, "px": b, "evx": c, "y": d, "py": e, "evy": f}
normallot = pd.DataFrame(x)

#Lotterien (plus 1000 bei outcomes) #within subject
# Lotterie mit grossen Outcomes
# x + 1000
abig = a + 1000
# px (gleich wie bei normal lotteries)
bbig = b
# evx = abig * bbig
cbig = abig * bbig
# Y = evx/py
dbig = cbig / (1 - bbig)
# py = 1 - px
ebig = 1 - bbig
# evy = y * py
fbig = dbig * ebig
#creating data frame
x = {"x": abig, "px": bbig, "evx": cbig, "y": dbig, "py": ebig, "evy": fbig}
biglot = pd.DataFrame(x)

#Lotterien mit rare event 
#Zufällige Zahlen 1 bis 10 x 30
ahard = np.random.choice(a = np.arange(1,11), size = 30, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bhard = np.random.choice(a = np.arange(0.1, 0.3, 0.05), size = 30, replace = True)

#EVx: x * px
chard = ahard * bhard

# Y = evx/py
dhard = chard / (1 - bhard)

#py: 1 - px
ehard = 1 - bhard

#Evy: y * py
fhard = dhard * ehard

#creating data frame for nornal Lotteries
x = {"x": ahard, "px": bhard, "evx": chard, "y": dhard, "py": ehard, "evy": fhard}
hardlot = pd.DataFrame(x)
hardlot

#Lotterien mit hohen outcomes und rare events
#Zufällige Zahlen 1 bis 10 x 30
abigh = abig

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bbigh = b

#EVx: x * px
cbigh = abigh * bbigh

# Y = evx/py
dbigh = cbigh / (1 - bbigh)

#py: 1 - px
ebigh = 1 - bbigh

#Evy: y * py
fbigh = dbigh * ebigh

#creating data frame for nornal Lotteries
x = {"x": abigh, "px": bbigh, "evx": cbigh, "y": dbigh, "py": ebigh, "evy": fbigh}
bighardlot = pd.DataFrame(x)

#Safe choice links vs rechts #between subject
#variieren der Anzeige

#create the dataframe
data = pd.DataFrame([])

#draw the stimuli


for t in range(n_trials):
    # put here things that change every trial
    
    # first event
    
    
    # second event
    
    # third event
    
    #save data to file
    for label in expInfo.keys():
        data[label] = expInfo[label]
    data.to_csv(fileName + '.csv')
    
#cleanup
mywin.close()
core.quit()

In [81]:
#Lotteriensammlung
import numpy as np
import pandas as pd
#Zufällige Zahlen 1 bis 10 x 30
a = np.random.choice(a = np.arange(1,11), size = 30, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px)
b = np.random.choice(a = np.arange(0.3, 0.7, 0.05), size = 30, replace = True)

#EVx: x * px
c = a * b

# Y = evx/py
d = c / (1 - b)

#py: 1 - px
e = 1 - b

#Evy: y * py
f = d * e

#creating data frame for nornal Lotteries
x = {"x": a, "px": b, "evx": c, "y": d, "py": e, "evy": f}
normallot = pd.DataFrame(x)
normallot

,x,px,evx,y,py,evy
0,10,0.65,6.50,18.571429,0.35,6.50
1,4,0.65,2.60,7.428571,0.35,2.60
2,5,0.55,2.75,6.111111,0.45,2.75
3,10,0.50,5.00,10.000000,0.50,5.00
4,10,0.45,4.50,8.181818,0.55,4.50
5,5,0.65,3.25,9.285714,0.35,3.25
6,1,0.60,0.60,1.500000,0.40,0.60
7,1,0.65,0.65,1.857143,0.35,0.65
8,2,0.45,0.90,1.636364,0.55,0.90
9,8,0.65,5.20,14.857143,0.35,5.20


In [82]:
# Lotterie mit grossen Outcomes
# x + 1000
abig = np.random.choice(a = np.arange(1,11), size = 30, replace = True) + 1000
# px (gleich wie bei normal lotteries)
bbig = b
# evx = abig * bbig
cbig = abig * bbig
# Y = evx/py
dbig = cbig / (1 - bbig)
# py = 1 - px
ebig = 1 - bbig
# evy = y * py
fbig = dbig * ebig
#creating data frame
x = {"x": abig, "px": bbig, "evx": cbig, "y": dbig, "py": ebig, "evy": fbig}
biglot = pd.DataFrame(x)
biglot

,x,px,evx,y,py,evy
0,1010,0.65,656.50,1875.714286,0.35,656.50
1,1004,0.65,652.60,1864.571429,0.35,652.60
2,1005,0.55,552.75,1228.333333,0.45,552.75
3,1010,0.50,505.00,1010.000000,0.50,505.00
4,1010,0.45,454.50,826.363636,0.55,454.50
5,1005,0.65,653.25,1866.428571,0.35,653.25
6,1001,0.60,600.60,1501.500000,0.40,600.60
7,1001,0.65,650.65,1859.000000,0.35,650.65
8,1002,0.45,450.90,819.818182,0.55,450.90
9,1008,0.65,655.20,1872.000000,0.35,655.20


In [87]:
# schwere Lotterie
#Zufällige Zahlen 1 bis 10 x 30
ahard = np.random.choice(a = np.arange(1,11), size = 30, replace = True)

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bhard = np.random.choice(a = np.arange(0.1, 0.3, 0.05), size = 30, replace = True)

#EVx: x * px
chard = ahard * bhard

# Y = evx/py
dhard = chard / (1 - bhard)

#py: 1 - px
ehard = 1 - bhard

#Evy: y * py
fhard = dhard * ehard

#creating data frame for nornal Lotteries
x = {"x": ahard, "px": bhard, "evx": chard, "y": dhard, "py": ehard, "evy": fhard}
hardlot = pd.DataFrame(x)
hardlot

,x,px,evx,y,py,evy
0,9,0.15,1.35,1.588235,0.85,1.35
1,7,0.10,0.70,0.777778,0.90,0.70
2,6,0.25,1.50,2.000000,0.75,1.50
3,7,0.25,1.75,2.333333,0.75,1.75
4,8,0.20,1.60,2.000000,0.80,1.60
5,3,0.10,0.30,0.333333,0.90,0.30
6,5,0.20,1.00,1.250000,0.80,1.00
7,6,0.15,0.90,1.058824,0.85,0.90
8,5,0.10,0.50,0.555556,0.90,0.50
9,4,0.10,0.40,0.444444,0.90,0.40


In [84]:
# schwere + big Lotterie
#Zufällige Zahlen 1 bis 10 x 30
abigh = abig

#Zufällige Wahrscheinlichkeiten x 30 (px zwieschen 0.1 und 0.3)
bbigh = b

#EVx: x * px
cbigh = abigh * bbigh

# Y = evx/py
dbigh = cbigh / (1 - bbigh)

#py: 1 - px
ebigh = 1 - bbigh

#Evy: y * py
fbigh = dbigh * ebigh

#creating data frame for nornal Lotteries
x = {"x": abigh, "px": bbigh, "evx": cbigh, "y": dbigh, "py": ebigh, "evy": fbigh}
bighardlot = pd.DataFrame(x)
bighardlot

,x,px,evx,y,py,evy
0,1010,0.65,656.50,1875.714286,0.35,656.50
1,1004,0.65,652.60,1864.571429,0.35,652.60
2,1005,0.55,552.75,1228.333333,0.45,552.75
3,1010,0.50,505.00,1010.000000,0.50,505.00
4,1010,0.45,454.50,826.363636,0.55,454.50
5,1005,0.65,653.25,1866.428571,0.35,653.25
6,1001,0.60,600.60,1501.500000,0.40,600.60
7,1001,0.65,650.65,1859.000000,0.35,650.65
8,1002,0.45,450.90,819.818182,0.55,450.90
9,1008,0.65,655.20,1872.000000,0.35,655.20
